**Dependencies**

In [23]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import numpy as np
import tensorflow as tf

from gensim.models import Word2Vec
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Input

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**Word2Vec Model**

In [ ]:
costco_df = pd.read_csv("costco_items_unique.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
stopwords = set([
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself',
    'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be',
    'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an',
    'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
    'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',
    'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
    'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all',
    'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just',
    'don', 'should', 'now'
])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def prepocess_dataset(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  tokens = [word for word in text.split() if word not in stopwords]
  return tokens

costco_df['New'] = costco_df['Item Name'].astype(str) + ' ' + costco_df['Item Description'].astype(str) + ' ' + costco_df['Category'].astype(str) + ' ' + costco_df['Price'].astype(str)
costco_df['Processed Description'] = costco_df['New'].apply(prepocess_dataset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
sentences = costco_df['Processed Description'].tolist()
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def similar_items(item_name, top_n=2):
    item_index = costco_df[costco_df['Item Name'] == item_name].index[0]
    item_vector = model.wv[model.wv.index_to_key[item_index]]
    similar_items = model.wv.similar_by_vector(item_vector, topn=top_n)
    similar_item_names = [costco_df.iloc[model.wv.key_to_index[item]].loc['Item Name'] for item, _ in similar_items]
    return similar_item_names

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
similar_items_list = similar_items("Gerber Organic Baby Food #3856")
print(similar_items_list[1])

Bounty Paper Towels #6755


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**FP-Growth Model**

In [ ]:
df_itemsets = pd.read_csv("costco_transactions_with_descriptions.csv")
transactions = df_itemsets.groupby('Transaction ID')['Item Name'].apply(list).tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def get_recommendations(cart_items, frequent_itemsets, rules):
    cart_df = pd.DataFrame([cart_items])
    cart_encoded = cart_df[cart_df.columns[cart_df.columns.isin(cart_items)]]
    cart_encoded = cart_encoded.fillna(0)

    itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: set(cart_items).issubset(x))]

    recommendations = []
    for itemset in itemsets['itemsets']:
        associated_items = rules[rules['antecedents'] == frozenset(itemset)]
        recommendations.extend(associated_items['consequents'])

    return set(recommendations) - set(cart_items)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
cart_items = ['Organic Blueberries', 'Organic Strawberries']
print(get_recommendations(cart_items, frequent_itemsets, rules))

{frozenset({'Kirkland Signature Organic Olive Oil'})}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**LSTM Model**

In [4]:
purchase_df = pd.read_csv("past.csv")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
purchase_df['Date'] = pd.to_datetime(purchase_df['Date'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
purchase_df = purchase_df.sort_values(by='Date')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
item_dict = {item: i for i, item in enumerate(purchase_df['Product Name'].unique())}
purchase_df['Product Name'] = purchase_df['Product Name'].map(item_dict)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
purchase_sequence = purchase_df['Product Name'].tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 50
padded_sequence = pad_sequences([purchase_sequence], maxlen=max_sequence_length, padding='post')[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
X = []
y = []
sequence_length = max_sequence_length - 1

for i in range(len(padded_sequence) - sequence_length):
    X.append(padded_sequence[i:i+sequence_length])
    y.append(padded_sequence[i+sequence_length])

X = np.array(X)
y = np.array(y)

num_items = len(item_dict)
y = np.eye(num_items)[y]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
if len(X) < 2:
    X_train, y_train = X, y
    X_val, y_val = X, y
else:
    split_index = int(0.8 * len(X))
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = Sequential([
    Embedding(input_dim=num_items, output_dim=50, input_length=sequence_length),
    LSTM(100, return_sequences=False),
    Dense(num_items, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 4.5729 - accuracy: 0.0000e+00 - val_loss: 4.5530 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 115ms/step - loss: 4.5530 - accuracy: 1.0000 - val_loss: 4.5322 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 99ms/step - loss: 4.5322 - accuracy: 1.0000 - val_loss: 4.5086 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 103ms/step - loss: 4.5086 - accuracy: 1.0000 - val_loss: 4.4806 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 100ms/step - loss: 4.4806 - accuracy: 1.0000 - val_loss: 4.4455 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 107ms/step - loss: 4.4455 - accuracy: 1.0000 - val_loss: 4.3996 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 109ms/step - loss: 4.3996 - accuracy: 1.0000 - val_loss: 4.3362 - val_accuracy: 1.0000
Epoch 8/10
1/1 [====

In [17]:
def get_lstm_recommendations(model, input_sequence, item_dict, top_n=5):
    input_sequence = np.array(input_sequence).reshape(1, -1)
    predictions = model.predict(input_sequence)
    predicted_items = np.argsort(predictions[0])[-top_n:]
    reverse_item_dict = {v: k for k, v in item_dict.items()}
    return [reverse_item_dict[i] for i in predicted_items]

test_sequence = padded_sequence[-5:]
recommendations = get_lstm_recommendations(model, test_sequence, item_dict)
print(f'Recommendations: {recommendations}')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1/1 [==============================] - 1s 804ms/step
Recommendations: ['Snacks', 'Garlic', 'Condiments', 'Toothpaste', 'Coffee Maker']


**Neural Collaborative Filtering**

In [18]:
df = pd.read_csv('past.csv')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
from sklearn.preprocessing import LabelEncoder

product_encoder = LabelEncoder()
store_encoder = LabelEncoder()

df['Product Name'] = product_encoder.fit_transform(df['Product Name'])
df['Store'] = store_encoder.fit_transform(df['Store'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
interaction_matrix = pd.pivot_table(df, index='Product Name', columns='Store', values='Price', fill_value=0)
user_item_matrix = interaction_matrix.values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
X_train, X_test = train_test_split(user_item_matrix, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
model = Sequential([
    Input(shape=(user_item_matrix.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(user_item_matrix.shape[1], activation='sigmoid')  # Output layer
])

model.compile(optimizer='adam', loss='binary_crossentropy')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
historyTwo = model.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2/2 [==============================] - 1s 229ms/step - loss: 61068.6055 - val_loss: 36370.7109
Epoch 2/10
2/2 [==============================] - 0s 70ms/step - loss: -7799.5474 - val_loss: -23729.9395
Epoch 3/10
2/2 [==============================] - 0s 61ms/step - loss: -82866.7188 - val_loss: -86857.6875
Epoch 4/10
2/2 [==============================] - 0s 42ms/step - loss: -159181.8750 - val_loss: -154068.6875
Epoch 5/10
2/2 [==============================] - 0s 37ms/step - loss: -233713.6250 - val_loss: -225867.4375
Epoch 6/10
2/2 [==============================] - 0s 67ms/step - loss: -321739.8750 - val_loss: -303964.4062
Epoch 7/10
2/2 [==============================] - 0s 46ms/step - loss: -410975.8750 - val_loss: -389878.3125
Epoch 8/10
2/2 [==============================] - 0s 59ms/step - loss: -512852.7812 - val_loss: -486851.8125
Epoch 9/10
2/2 [==============================] - 0s 41ms/step - loss: -628930.6875 - val_loss: -596728.8750
Epoch 10/10
2/2 [=====================

In [26]:
predictionsTwo = model.predict(user_item_matrix)

4/4 [==============================] - 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
def recommend_items(predictions, top_n=10):
    user_predictions = predictions[0]
    top_indices = np.argsort(user_predictions)[::-1][:top_n]
    return top_indices

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
def index_to_product_name(indices, encoder):
    return encoder.inverse_transform(indices)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
top_n = 1
top_indices = recommend_items(predictionsTwo, top_n)

recommended_product_names = index_to_product_name(top_indices, product_encoder)

for product in recommended_product_names:
    print(product)

Butter


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
